---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import collections
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [5]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [6]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()
  
text = read_data(filename)
text2 = zip(text, text[1:])
print(text2[:20])
print('Data2 size %d' % len(text2))
del text

[(' ', 'a'), ('a', 'n'), ('n', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'h'), ('h', 'i'), ('i', 's'), ('s', 'm'), ('m', ' '), (' ', 'o'), ('o', 'r'), ('r', 'i'), ('i', 'g'), ('g', 'i'), ('i', 'n'), ('n', 'a'), ('a', 't'), ('t', 'e'), ('e', 'd')]
Data2 size 99999999


Create a small validation set.

In [7]:
valid_size = 1000
valid_text = text2[:valid_size]
train_text = text2[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:32])
print(valid_size, valid_text[:32])

99998999 [('o', 'n'), ('n', 's'), ('s', ' '), (' ', 'a'), ('a', 'n'), ('n', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'h'), ('h', 'i'), ('i', 's'), ('s', 't'), ('t', 's'), ('s', ' '), (' ', 'a'), ('a', 'd'), ('d', 'v'), ('v', 'o'), ('o', 'c'), ('c', 'a'), ('a', 't'), ('t', 'e'), ('e', ' '), (' ', 's'), ('s', 'o'), ('o', 'c'), ('c', 'i'), ('i', 'a'), ('a', 'l'), ('l', ' '), (' ', 'r'), ('r', 'e')]
1000 [(' ', 'a'), ('a', 'n'), ('n', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'h'), ('h', 'i'), ('i', 's'), ('s', 'm'), ('m', ' '), (' ', 'o'), ('o', 'r'), ('r', 'i'), ('i', 'g'), ('g', 'i'), ('i', 'n'), ('n', 'a'), ('a', 't'), ('t', 'e'), ('e', 'd'), ('d', ' '), (' ', 'a'), ('a', 's'), ('s', ' '), (' ', 'a'), ('a', ' '), (' ', 't'), ('t', 'e'), ('e', 'r'), ('r', 'm'), ('m', ' '), (' ', 'o')]


Utility functions to map characters to vocabulary IDs and back.

In [8]:
letter_count = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])
vocabulary_size = letter_count**2

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char[0]) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0

def chars2id(chars):
    return char2id(chars[0])*letter_count + char2id(chars[1])

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

def id2chars(dictid):
    c1 = id2char(dictid // letter_count)
    c2 = id2char(dictid % letter_count)
    return (c1, c2)

print(chars2id((' ', 'a')), chars2id(('b', 'z')))
print(id2chars(1), id2chars(80))

1 80
(' ', 'a') ('b', 'z')


2-gram embedding

Function to generate a training batch for the LSTM model.
  * *num_unrollings* defines number of LSTM cells

In [9]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        # Segment is number of batches in the text
        segment = self._text_size // batch_size
        # Cursor is a list of pointers set to the beggining of each segment
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b] = chars2id(self._text[self._cursor[b]])
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

    
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

b = train_batches.next()
print(b[0])

[ 419.  629.  336.    1.  352.  221.  675.   52.  549.  360.  383.  221.
  135.  137.  405.  680.  423.   27.   20.   45.  263.    1.  549.  133.
  162.   47.  135.  149.   40.  508.  258.  405.   27.  197.  257.  501.
   82.   46.    4.  366.  540.  567.  135.  405.  411.  513.  309.  153.
  640.  155.  548.  155.   19.  262.  549.  108.  548.  149.  167.  129.
  558.   47.   43.  522.]


In [7]:
# def logprob(predictions, labels):
#     """Log-probability of the true labels in a predicted batch."""
#     predictions[predictions < 1e-10] = 1e-10
#     return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

# def sample_distribution(distribution):
#     """Sample one element from a distribution assumed to be an array of normalized
#     probabilities.
#     """
#     r = random.uniform(0, 1)
#     s = 0
#     for i in range(len(distribution)):
#         s += distribution[i]
#         if s >= r:
#             return i
#     return len(distribution) - 1

# def sample(prediction):
#     """Turn a (column) prediction into 1-hot encoded samples."""
#     p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
#     p[0, sample_distribution(prediction[0])] = 1.0
#     return p

# def random_distribution():
#     """Generate a random column of probabilities."""
#     b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
#     return b/np.sum(b, 1)[:,None]

# print('sample_distribution([0.1, 0.2, 0.3, 0.4]) = %d' % sample_distribution([0.1, 0.2, 0.3, 0.4]))
# random_dist = random_distribution()
# print('random_dist = ', random_dist)
# print('sample(random_dist) = ', sample(random_dist))

sample_distribution([0.1, 0.2, 0.3, 0.4]) = 2
random_dist =  [[ 0.01386932  0.01533595  0.08415141  0.03484591  0.0203517   0.06124168
   0.02061342  0.03298343  0.0007279   0.0109498   0.00705117  0.0801856
   0.07798728  0.02601601  0.03483116  0.06856687  0.05256913  0.0028407
   0.08185048  0.06418326  0.03366877  0.08387608  0.00085087  0.04349868
   0.01351004  0.00875995  0.02468342]]
sample(random_dist) =  [[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Simple LSTM Model.

  * Training data is a list of matrices with input data. 
  * The size of the list equals to number of LSTM cells.
  * The input is provided as 1-hot encoded letter (1-gram). The size of the 1-hot vector is vocabulary_size
  * Each matrix has dimension batch_size x vocabulary_size
  
What is  num_nodes?

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [28]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294305 learning rate: 10.000000
Minibatch perplexity: 26.96
 priusuguf   legoszwzqzaz rhwrpd a y tplgihnhftndfri  x g ezcspch adrsm anaxtzdt
cneevnz rqziwotg itzsc tqc  hptor c splph iroxeoeiuvcycign  xzr  dgp i  ioxiad w
xqdutabsteid  izj y vlnewdaa  ixqfd htfdvpaxayh wjfs ienziwnt nnssdfimicvxb  lii
sannreejh r iigleyvtnaoy ronrebrj y i a eao i e sgr wzezgedcvo rntrws  qndklno n
fe elcjnzanr emjffvzzyea yk tie  jdnihr tsnkdnt pai  tpatisdujit  nzxao mpt apat
Validation set perplexity: 19.92
Average loss at step 100: 2.595111 learning rate: 10.000000
Minibatch perplexity: 10.44
Validation set perplexity: 10.44
Average loss at step 200: 2.260032 learning rate: 10.000000
Minibatch perplexity: 7.91
Validation set perplexity: 8.69
Average loss at step 300: 2.100649 learning rate: 10.000000
Minibatch perplexity: 7.52
Validation set perplexity: 7.75
Average loss at step 400: 1.995700 learning rate: 10.000000
Minibatch perplexity: 6.87
Validation set per